In [4]:
import pandas as pd
import numpy as np
import sqlalchemy as sqlalchemy
import psycopg2
from psycopg2.extras import execute_values
from sqlalchemy import create_engine
from pathlib import Path
import time
from datetime import datetime
from datetime import timedelta
import os
import glob
import warnings
import re
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
pd.options.mode.chained_assignment = None  # default='warn

# Создаем соединение с базой данных
conn = psycopg2.connect(dbname="", host="",
                        user="a.tretyakov", password="admin")

folder_path_fact = r'S:\\BI\\data\\face_id_ano\\fact'

# Путь к файлу для замены наименований должностей
replacement = pd.read_excel(r'S:\Должности.xlsx')

# Функция для чтения папок с excel-файлами
def find_excel_files(root_dir):
    # Список для хранения путей к Excel файлам
    excel_files = []
    
    # Перебор всех файлов и каталогов в текущем каталоге
    for dirpath, dirnames, filenames in os.walk(root_dir):
        # Перебор всех файлов в текущем каталоге
        for filename in [f for f in filenames if f.endswith('.xlsx') or f.endswith('.xls')]:
            # Получение полного пути к файлу
            file_path = os.path.join(dirpath, filename)
            excel_files.append(file_path)
    
    # Пробегаю по итоговому списку и удаляю элементые, которые редактируются.
    for i in excel_files:
        if "~$" in i.lower():
            excel_files.remove(i)
    
    return excel_files

# Датафрейм с путями к файлам Excel по объектам. Поиск файлов Excel
excel_files = find_excel_files(folder_path_fact)
excel_files

['S:\\\\22. Офис управления проектами\\\\Общая документация\\\\10. Аналитика\\\\BI\\\\data\\\\face_id_ano\\\\fact\\2024-08-06.xls',
 'S:\\\\22. Офис управления проектами\\\\Общая документация\\\\10. Аналитика\\\\BI\\\\data\\\\face_id_ano\\\\fact\\2024-08-08.xls',
 'S:\\\\22. Офис управления проектами\\\\Общая документация\\\\10. Аналитика\\\\BI\\\\data\\\\face_id_ano\\\\fact\\2024-08-09-2024-08-11.xls',
 'S:\\\\22. Офис управления проектами\\\\Общая документация\\\\10. Аналитика\\\\BI\\\\data\\\\face_id_ano\\\\fact\\2024-08-13.xls',
 'S:\\\\22. Офис управления проектами\\\\Общая документация\\\\10. Аналитика\\\\BI\\\\data\\\\face_id_ano\\\\fact\\2024-08-14.xls',
 'S:\\\\22. Офис управления проектами\\\\Общая документация\\\\10. Аналитика\\\\BI\\\\data\\\\face_id_ano\\\\fact\\2024-08-15.xls',
 'S:\\\\22. Офис управления проектами\\\\Общая документация\\\\10. Аналитика\\\\BI\\\\data\\\\face_id_ano\\\\fact\\2024-08-19.xls',
 'S:\\\\22. Офис управления проектами\\\\Общая документация\\\\10

In [5]:
# Функция обработки excel-файлов 
def read_and_process_excel(file_path):    
    df = pd.read_excel(file_path, header=None)
    df.rename(columns={0: 'date', 1: 'otdel', 2: 'tabel_number', 3: 'post', 4: 'entry', 5: 'exit'}, inplace=True) # Имена колонок
    df = df[df.exit != 'уход' ].dropna(subset=['entry', 'exit']).fillna(method='ffill').reset_index() # Удаление пустых строк по столбцам вход и выход, заполнение вниз, сброс индекса
    df = df[(df.entry != '(нет)') & (df.exit != '(нет)')] 
    df['entry'] = df['entry'].str[:5]
    df['exit'] = df['exit'].str[:5]
    df['work_time'] = pd.to_datetime(df['date'] + ' ' + df['exit']) - pd.to_datetime(df['date'] + ' ' + df['entry'])
    df = df.groupby(['date', 'otdel', 'tabel_number', 'post'], as_index=False).agg({'work_time': 'sum'})
    df['work_time'] = df['work_time'] / np.timedelta64(1, 'h') # Переводим дни-часы в количество часов
    df['post'] = df['post'].str.strip()
    return df

# Проверка и объединение файлов с объектами
def process_excel_files(excel_files):

    all_dfs = []
    for file_path in excel_files:
        # Проверяем, существует ли файл
        if not os.path.exists(file_path):
            print(f"Файл '{file_path}' не найден.")
            continue

        # Читаем данные из Excel файла
        df = read_and_process_excel(file_path)

        if df is not None:
            all_dfs.append(df)

    # Объединяем DataFrames
    if len(all_dfs) > 0:
        combined_df = pd.concat(all_dfs)
        return combined_df
    else:
        return None
    
combined_df_fact = process_excel_files(excel_files)
combined_df_fact['obj_key'] = '020-0682'

combined_df_fact

,date,otdel,tabel_number,post,work_time,obj_key
0,2024-08-06,AKVASTROY,AKVA STROY,бригадир,10.450000,020-0682
1,2024-08-06,AKVASTROY,AKVASTROY,бригадир,10.766667,020-0682
2,2024-08-06,BEKCI(ОХРАНА),5506,BEKCI(ОХРАНА),11.933333,020-0682
3,2024-08-06,BEKCI(ОХРАНА),7826,BEKCI(ОХРАНА),0.433333,020-0682
4,2024-08-06,BEKCI(ОХРАНА),8678,BEKCI(ОХРАНА),11.650000,020-0682
...,...,...,...,...,...,...
935,2024-09-04,YEMEKHANE(СТОЛОВОЯ),9303,OFIS HIZMETI,9.566667,020-0682
936,2024-09-04,YEMEKHANE(СТОЛОВОЯ),9485,YEMAK HONA IŞÇSI,12.400000,020-0682
937,2024-09-04,YEMEKHANE(СТОЛОВОЯ),9571,YEMAK HONA IŞÇSI,9.750000,020-0682
938,2024-09-04,YEMEKHANE(СТОЛОВОЯ),9597,YEMAK HONA IŞÇSI,11.033333,020-0682


In [13]:
# Создаем соединение с базой данных


if combined_df_fact is not None:
    print('Объединенный DataFrame успешно создан.')
    
    rows_to_insert = [tuple(x) for x in combined_df_fact.to_numpy()]
    # Формирование SQL-запроса для вставки данных
    insert_query = "INSERT INTO anodb.dwh_raw.face_id_test (date, otdel, tabel_number, post, work_time, obj_key) VALUES %s;"
    cursor = conn.cursor()
    # Удаляем все данные из таблицы
    cursor.execute("TRUNCATE TABLE anodb.dwh_raw.face_id_test;")
    execute_values(cursor, insert_query, rows_to_insert)
    conn.commit()
    cursor.close()
    conn.close()
    print('\n Объединенный DataFrame успешно загружен в БД: anodb.dwh_raw.face_id_test')
else:
    print("Ни один из файлов не был успешно обработан.")

Объединенный DataFrame успешно создан.

 Объединенный DataFrame успешно загружен в БД: anodb.dwh_raw.face_id_test


# Создаю датафрейм план-факт по дням

In [7]:
# Создаю датафрейм факта по дням. Группирую по кодуДС и месяцу. Суммирую значения по "видам" работ.
df_fact_day = combined_df_fact.groupby(['obj_key', 'date'], as_index=False).agg({'work_time': 'mean', 'tabel_number': 'count'})
df_fact_day['date'] = pd.to_datetime(df_fact_day['date'])

In [8]:
# Парсинг плана
df2 = pd.read_excel(r'S:\\22. Офис управления проектами\\Общая документация\\10. Аналитика\\BI\\data\\face_id_ano\\plan\\07082024_Многопрофильный комплекс ГБУЗ ДГКБ св. Владимира ДЗМ.XLSX', header=1)
df2.drop(df2.columns[[1, 2, 3, 4]], axis= 1 , inplace= True )
df2 = df2.rename(columns={'020-0682 Многопрофильный комплекс ГБУЗ "ДГКБ св.Владимира ДЗМ" по адресу: ул. Рубцовско-Дворцовая, д. 1/3 (проектирование и строительство)"':'task'})
df_plan = pd.melt(df2, id_vars='task',  var_name="month", value_name="value")
df_plan['obj_key'] = '020-0682'

C:\Users\a.tretyakov\AppData\Local\Temp\ipykernel_18260\849707446.py:5: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df_plan = pd.melt(df2, id_vars='task',  var_name="month", value_name="value")


# Создам датафрейм сгруппированный по датам. Удаляю разбивку по работам, так как пока нет справочника для свода с должностями. Пока будем смотреть суммарно в разрезе дня

In [9]:
# Группирую по кодуДС и месяцу. Суммирую значения по "видам" работ
df_plan_month = df_plan.groupby(['obj_key', 'month'], as_index=False).agg({'value': 'sum'})

# Ищу минимальную и максимальную (+30 дней, т.к. 1 число месяца) даты. Затем создаю новый датафрейм со всеми датами между min и max.
dates = pd.date_range(df_plan_month['month'].min(), df_plan_month['month'].max() + timedelta(days=30))
new_df = pd.DataFrame({'date': dates})
df_plan_day = pd.merge(new_df.assign(grouper=new_df['date'].dt.to_period('M')),
               df_plan_month.assign(grouper=df_plan_month['month'].dt.to_period('M')),
               how='left', on='grouper')
df_plan_day.drop(['grouper', 'month'], axis= 1 , inplace= True )
plan_fact = pd.merge(df_plan_day, df_fact_day, how='left', left_on=['obj_key', 'date'], right_on = ['obj_key','date'])
plan_fact.rename(columns = {'value':'plan', 'tabel_number':'fact'}, inplace = True)

In [10]:
plan_fact

,date,obj_key,plan,work_time,fact
0,2022-01-01,020-0682,25.0,NaN,NaN
1,2022-01-02,020-0682,25.0,NaN,NaN
2,2022-01-03,020-0682,25.0,NaN,NaN
3,2022-01-04,020-0682,25.0,NaN,NaN
4,2022-01-05,020-0682,25.0,NaN,NaN
...,...,...,...,...,...
1456,2025-12-27,020-0682,23.0,NaN,NaN
1457,2025-12-28,020-0682,23.0,NaN,NaN
1458,2025-12-29,020-0682,23.0,NaN,NaN
1459,2025-12-30,020-0682,23.0,NaN,NaN


In [15]:
conn = psycopg2.connect(dbname="anodb", host="192.168.89.32",
                        user="a.tretyakov", password="admin")

if plan_fact is not None:
    print('Объединенный DataFrame План-Факт успешно создан.')
    
    rows_to_insert = [tuple(x) for x in plan_fact.to_numpy()]
    # Формирование SQL-запроса для вставки данных
    insert_query = "INSERT INTO anodb.dwh_data.face_id_plan_fact (date, obj_key, plan, work_time, fact) VALUES %s;"
    cursor = conn.cursor()
    # Удаляем все данные из таблицы
    cursor.execute("TRUNCATE TABLE anodb.dwh_data.face_id_plan_fact;")
    execute_values(cursor, insert_query, rows_to_insert)
    conn.commit()
    cursor.close()
    conn.close()
    print('\n Объединенный DataFrame успешно загружен в БД: anodb.dwh_data.face_id_plan_fact')
else:
    print("Ни один из файлов не был успешно обработан.")

Объединенный DataFrame План-Факт успешно создан.

 Объединенный DataFrame успешно загружен в БД: anodb.dwh_data.face_id_plan_fact


In [63]:
plan_fact.to_excel(r'S:\\22. Офис управления проектами\\Общая документация\\10. Аналитика\\Сервисы\\RFID_метки\\data_face_id_ano\\test_face_id_ano_2.xlsx')